In [1]:
import pandas as pd
import json
import os
import re
import matplotlib.pyplot as plt
import numpy as np
import datetime as dt

In [2]:
instagramFileName = '/Users/amanshah/Documents/Software Projects/Instagram-Analysis/message_1.json'
data_path = "C:/Users/etc.json"
with open(instagramFileName) as file:
    json_data = json.load(file)
    print(json_data)

{'participants': [{'name': 'Sumanth Kumar'}, {'name': 'Dew Bhaumik'}, {'name': 'Aman'}, {'name': 'Lokesh Patel'}, {'name': 'Suhani Trivedi'}, {'name': 'waris.z_'}], 'messages': [{'sender_name': 'Sumanth Kumar', 'timestamp_ms': 1656496133346, 'content': 'itâ\x80\x99s 6am', 'type': 'Generic', 'is_unsent': False, 'is_taken_down': False, 'bumped_message_metadata': {'bumped_message': 'itâ\x80\x99s 6am', 'is_bumped': False}}, {'sender_name': 'Sumanth Kumar', 'timestamp_ms': 1656496127668, 'content': 'why r u awake', 'type': 'Generic', 'is_unsent': False, 'is_taken_down': False, 'bumped_message_metadata': {'bumped_message': 'why r u awake', 'is_bumped': False}}, {'sender_name': 'waris.z_', 'timestamp_ms': 1656495009020, 'content': 'I found Dewâ\x80\x99s girl', 'type': 'Generic', 'is_unsent': False, 'is_taken_down': False, 'bumped_message_metadata': {'bumped_message': 'I found Dewâ\x80\x99s girl', 'is_bumped': False}}, {'sender_name': 'waris.z_', 'timestamp_ms': 1656495008985, 'photos': [{'uri

In [3]:
df = pd.DataFrame(json_data["messages"], columns = ['timestamp_ms', 'sender_name', 'content', 'is_unsent', 'type'])
df.rename(columns={"timestamp_ms": "Time", "sender_name": "Name", "content": "Messages", "is_unsent": "Unsent", "type": "Type"}, inplace=True)
print("len of the dataframe is ",len(df))
df


len of the dataframe is  2062


,Time,Name,Messages,Unsent,Type
0,1656496133346,Sumanth Kumar,itâs 6am,False,Generic
1,1656496127668,Sumanth Kumar,why r u awake,False,Generic
2,1656495009020,waris.z_,I found Dewâs girl,False,Generic
3,1656495008985,waris.z_,NaN,False,Generic
4,1656484488193,waris.z_,waris.z_ sent an attachment.,False,Share
...,...,...,...,...,...
2057,1631655987285,Sumanth Kumar,howâs life,False,Generic
2058,1631655951325,Lokesh Patel,Helllooooo,False,Generic
2059,1631655941155,Sumanth Kumar,hi,False,Generic
2060,1631655805525,Aman,Hey guys,False,Generic


In [4]:
df['Time'] = pd.to_datetime(df['Time'], unit='ms', errors = 'coerce')
for c in df:
    if c == 'Time': 
        df[c] = pd.DatetimeIndex(pd.to_datetime(
            df[c],
            format='%Y-%m-%d %H:%M:%S')).tz_localize(
                'UTC').tz_convert(
                    'US/Eastern')

df['Time'] = df['Time'].apply(lambda x: x.strftime('%Y-%m-%d %I:%M:%S '))

df

,Time,Name,Messages,Unsent,Type
0,2022-06-29 05:48:53,Sumanth Kumar,itâs 6am,False,Generic
1,2022-06-29 05:48:47,Sumanth Kumar,why r u awake,False,Generic
2,2022-06-29 05:30:09,waris.z_,I found Dewâs girl,False,Generic
3,2022-06-29 05:30:08,waris.z_,NaN,False,Generic
4,2022-06-29 02:34:48,waris.z_,waris.z_ sent an attachment.,False,Share
...,...,...,...,...,...
2057,2021-09-14 05:46:27,Sumanth Kumar,howâs life,False,Generic
2058,2021-09-14 05:45:51,Lokesh Patel,Helllooooo,False,Generic
2059,2021-09-14 05:45:41,Sumanth Kumar,hi,False,Generic
2060,2021-09-14 05:43:25,Aman,Hey guys,False,Generic


In [5]:
df = df.replace(['waris.z_'],'Waris')
df = df.replace(['Lokesh Patel'],'Lokesh')
df = df.replace(['Dew Bhaumik'],'Dew')
df = df.replace(['Suhani Trivedi'],'Suhani')
df = df.replace(['Sumanth Kumar'],'Sumanth')

for x in df.index:
    if df.loc[x, 'Type'] == "Share":
        df = df.replace('Share',np.NaN)
    elif df.loc[x, "Unsent"] == "True":
        df = df.replace('True',np.NaN)

df

,Time,Name,Messages,Unsent,Type
0,2022-06-29 05:48:53,Sumanth,itâs 6am,False,Generic
1,2022-06-29 05:48:47,Sumanth,why r u awake,False,Generic
2,2022-06-29 05:30:09,Waris,I found Dewâs girl,False,Generic
3,2022-06-29 05:30:08,Waris,NaN,False,Generic
4,2022-06-29 02:34:48,Waris,waris.z_ sent an attachment.,False,NaN
...,...,...,...,...,...
2057,2021-09-14 05:46:27,Sumanth,howâs life,False,Generic
2058,2021-09-14 05:45:51,Lokesh,Helllooooo,False,Generic
2059,2021-09-14 05:45:41,Sumanth,hi,False,Generic
2060,2021-09-14 05:43:25,Aman,Hey guys,False,Generic


In [6]:
df = df.dropna()
df

,Time,Name,Messages,Unsent,Type
0,2022-06-29 05:48:53,Sumanth,itâs 6am,False,Generic
1,2022-06-29 05:48:47,Sumanth,why r u awake,False,Generic
2,2022-06-29 05:30:09,Waris,I found Dewâs girl,False,Generic
6,2022-06-28 02:15:45,Sumanth,ok and?,False,Generic
7,2022-06-28 02:13:04,Waris,Unavailable,False,Generic
...,...,...,...,...,...
2057,2021-09-14 05:46:27,Sumanth,howâs life,False,Generic
2058,2021-09-14 05:45:51,Lokesh,Helllooooo,False,Generic
2059,2021-09-14 05:45:41,Sumanth,hi,False,Generic
2060,2021-09-14 05:43:25,Aman,Hey guys,False,Generic


In [7]:
authors = ['Sumanth' , 'Waris', 'Aman', 'Lokesh', 'Dew', 'Suhani']
df = df[df['Name'].isin(authors)]
df

,Time,Name,Messages,Unsent,Type
0,2022-06-29 05:48:53,Sumanth,itâs 6am,False,Generic
1,2022-06-29 05:48:47,Sumanth,why r u awake,False,Generic
2,2022-06-29 05:30:09,Waris,I found Dewâs girl,False,Generic
6,2022-06-28 02:15:45,Sumanth,ok and?,False,Generic
7,2022-06-28 02:13:04,Waris,Unavailable,False,Generic
...,...,...,...,...,...
2057,2021-09-14 05:46:27,Sumanth,howâs life,False,Generic
2058,2021-09-14 05:45:51,Lokesh,Helllooooo,False,Generic
2059,2021-09-14 05:45:41,Sumanth,hi,False,Generic
2060,2021-09-14 05:43:25,Aman,Hey guys,False,Generic


In [8]:
df['number_of_words'] = df.Messages.apply(lambda x: len(x.split()))
df['number_of_characters'] = df['Messages'].str.len()
df

,Time,Name,Messages,Unsent,Type,number_of_words,number_of_characters
0,2022-06-29 05:48:53,Sumanth,itâs 6am,False,Generic,2,10
1,2022-06-29 05:48:47,Sumanth,why r u awake,False,Generic,4,13
2,2022-06-29 05:30:09,Waris,I found Dewâs girl,False,Generic,4,20
6,2022-06-28 02:15:45,Sumanth,ok and?,False,Generic,2,7
7,2022-06-28 02:13:04,Waris,Unavailable,False,Generic,1,11
...,...,...,...,...,...,...,...
2057,2021-09-14 05:46:27,Sumanth,howâs life,False,Generic,2,12
2058,2021-09-14 05:45:51,Lokesh,Helllooooo,False,Generic,1,10
2059,2021-09-14 05:45:41,Sumanth,hi,False,Generic,1,2
2060,2021-09-14 05:43:25,Aman,Hey guys,False,Generic,2,8


In [9]:
sumanth_df = df.loc[df['Name'] == 'Sumanth']
dew_df = df.loc[df['Name'] == 'Dew']
lokesh_df = df.loc[df['Name'] == 'Lokesh']
suhani_df = df.loc[df['Name'] == 'Suhani']
Waris_df = df.loc[df['Name'] == 'Waris']
aman_df = df.loc[df['Name'] == 'Aman']
sumanth_df

,Time,Name,Messages,Unsent,Type,number_of_words,number_of_characters
0,2022-06-29 05:48:53,Sumanth,itâs 6am,False,Generic,2,10
1,2022-06-29 05:48:47,Sumanth,why r u awake,False,Generic,4,13
6,2022-06-28 02:15:45,Sumanth,ok and?,False,Generic,2,7
8,2022-06-28 10:07:25,Sumanth,mad jelaous rn,False,Generic,3,14
10,2022-06-27 11:03:32,Sumanth,for aman,False,Generic,2,8
...,...,...,...,...,...,...,...
2051,2021-09-14 05:47:37,Sumanth,ur virus,False,Generic,2,8
2052,2021-09-14 05:47:33,Sumanth,ððð,False,Generic,1,12
2054,2021-09-14 05:47:17,Sumanth,wish i was ranchodas tho,False,Generic,5,24
2057,2021-09-14 05:46:27,Sumanth,howâs life,False,Generic,2,12


In [10]:
dew_df

,Time,Name,Messages,Unsent,Type,number_of_words,number_of_characters
12,2022-06-27 12:30:04,Dew,Cock fr,False,Generic,2,7
14,2022-06-27 12:17:39,Dew,Well it wasn't small when it was in me yesterd...,False,Generic,11,54
15,2022-06-27 12:17:29,Dew,Ahhh,False,Generic,1,4
17,2022-06-27 12:16:53,Dew,When u look down can u see ur penis or does ur...,False,Generic,16,68
20,2022-06-27 12:16:36,Dew,Like deadass,False,Generic,2,12
...,...,...,...,...,...,...,...
2031,2021-09-14 05:50:50,Dew,Ass,False,Generic,1,3
2034,2021-09-14 05:50:40,Dew,U don't know what that meansð,False,Generic,6,32
2036,2021-09-14 05:49:55,Dew,Urs is bigger,False,Generic,3,13
2038,2021-09-14 05:49:37,Dew,Kem cho maru nam dhagri che,False,Generic,6,27


In [11]:
lokesh_df

,Time,Name,Messages,Unsent,Type,number_of_words,number_of_characters
124,2022-06-24 10:27:38,Lokesh,Ur mad coz bad,False,Generic,4,14
128,2022-06-24 10:27:22,Lokesh,Because Iâm the one,False,Generic,4,21
130,2022-06-24 10:27:19,Lokesh,Makes sense tbh,False,Generic,3,15
132,2022-06-24 10:27:09,Lokesh,So hemadri not the one yo,False,Generic,6,25
148,2022-06-24 10:25:21,Lokesh,ðð,False,Generic,1,8
...,...,...,...,...,...,...,...
2047,2021-09-14 05:47:58,Lokesh,#golddiggervibes,False,Generic,1,16
2049,2021-09-14 05:47:48,Lokesh,Iâd take chatur as my husband,False,Generic,6,31
2050,2021-09-14 05:47:41,Lokesh,Fuck chatur balling tho,False,Generic,4,23
2056,2021-09-14 05:46:55,Lokesh,Pretty sad feeling like raju rostogi rn,False,Generic,7,39


In [12]:
suhani_df

,Time,Name,Messages,Unsent,Type,number_of_words,number_of_characters
114,2022-06-25 02:53:19,Suhani,itâs litterally just gonna get worse,False,Generic,6,38
115,2022-06-25 02:53:10,Suhani,We saw this coming,False,Generic,4,18
251,2022-06-24 01:24:37,Suhani,its good to do research but donât spread mis...,False,Generic,9,58
253,2022-06-24 01:24:09,Suhani,What this means is that every state can create...,False,Generic,89,477
391,2022-06-24 01:06:21,Suhani,You should. This isnt funny. Itâs rly import...,False,Generic,16,92
...,...,...,...,...,...,...,...
1989,2021-09-14 05:59:29,Suhani,yea line conversion s,False,Generic,4,21
1990,2021-09-14 05:59:26,Suhani,they skip so many steps,False,Generic,5,23
1992,2021-09-14 05:59:20,Suhani,the conversions are taking me so long to under...,False,Generic,9,51
1994,2021-09-14 05:59:12,Suhani,lmao first lecture,False,Generic,3,18


In [13]:
Waris_df

,Time,Name,Messages,Unsent,Type,number_of_words,number_of_characters
2,2022-06-29 05:30:09,Waris,I found Dewâs girl,False,Generic,4,20
7,2022-06-28 02:13:04,Waris,Unavailable,False,Generic,1,11
30,2022-06-27 12:15:06,Waris,It was truly a period of time,False,Generic,7,29
32,2022-06-27 12:14:21,Waris,Homie has no other jokes ððð¤£,False,Generic,6,37
33,2022-06-27 12:14:15,Waris,I know im doing good with my roasts when suman...,False,Generic,15,76
...,...,...,...,...,...,...,...
1760,2022-01-18 03:24:56,Waris,OGRE,False,Generic,1,4
1785,2022-01-11 07:11:57,Waris,Wtf,False,Generic,1,3
1800,2021-10-29 12:02:35,Waris,Ya weâre prolly just dumb,False,Generic,5,27
1801,2021-10-29 12:02:30,Waris,Lmao,False,Generic,1,4


In [14]:
aman_df

,Time,Name,Messages,Unsent,Type,number_of_words,number_of_characters
88,2022-06-27 12:03:28,Aman,You went there ð¬,False,Generic,4,19
89,2022-06-27 12:03:17,Aman,Ohhhhh damn,False,Generic,2,11
122,2022-06-24 10:29:10,Aman,Making dosa to punish him every time he says h...,False,Generic,12,61
123,2022-06-24 10:28:51,Aman,Fam his mom is gonna lock his marriage with so...,False,Generic,13,66
126,2022-06-24 10:27:26,Aman,^^,False,Generic,1,2
...,...,...,...,...,...,...,...
2039,2021-09-14 05:49:25,Aman,Lmao,False,Generic,1,4
2053,2021-09-14 05:47:29,Aman,Nah you are Chatur,False,Generic,4,18
2055,2021-09-14 05:47:06,Aman,Lmao,False,Generic,1,4
2060,2021-09-14 05:43:25,Aman,Hey guys,False,Generic,2,8
